# Sklearn教程

In [ ]:
#如果你还没有安装sklearn可以运行这行代码
!pip install sklearn

## 1.监督学习Supervised Learning
### 1.1 广义线性模型 Linear Model
主要讲述一些用于回归的方法，其中目标值 y 是输入变量 x 的线性组合。 数学概念表示为：如果 $\hat{y}$ 是预测值，那么有：
$$
\hat{y}(w, x) = w_0 + w_1 x_1 + ... + w_p x_p
$$

在整个模块中，我们定义向量 w = (w_1,..., w_p) 作为 coef_ ，定义 w_0 作为 intercept_ 


#### 1.1.1 普通最小二乘法 Ordinary Least Squares
`LinearRegression`拟合一个带有系数$w = (w_1, ..., w_p) $的线性模型，使数据集实际观测数据和预测数据（估计值）之间的残差平方和最小。其数学表达式为:
$$
\underset{w}{min\,} {|| X w - y||_2}^2
$$

<img src='../pics/sphx_glr_plot_ols_001.png' width='50%'>

LinearRegression 会调用`fit`方法来拟合数组 X， y，并且将线性模型的系数 `w`存储在其成员变量 coef_ 中:

In [4]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])
reg.coef_

array([0.5, 0.5])

然而，对于普通最小二乘的系数估计问题，其依赖于模型各项的相互独立性。当各项是相关的，且设计矩阵 `X` 的各列近似线性相关，那么，设计矩阵会趋向于奇异矩阵，这种特性导致最小二乘估计对于随机误差非常敏感，可能产生很大的方差。例如，在没有实验设计的情况下收集到的数据，这种多重共线性（multicollinearity）的情况可能真的会出现。

**普通最小二乘法的复杂度**  
该方法使用X的奇异值分解来计算最小二乘解。如果X是一个形状为`(x_samples, n_features)`的矩阵，设$n_{sample}\geq_n{feature}$, 则该方法的复杂度为$O(n_{samples}n_{smaples}^2)$

### 1.1.2 岭回归
Rige回归通过对系数的大小施加惩罚来解决普通最小二乘法的一些问题。岭系数最小化的是带惩罚项的残差平方和，
$$
\underset{w}{min\,} {{|| X w - y||_2}^2 + \alpha {||w||_2}^2}
$$

其中，`α≥0`是控制系数收缩量的复杂性参数：α的值越大，收缩量越大，模型对共线性的鲁棒性也更强 
<img src='../pics/reg-2.jpeg' wifth='60%'>

与其他线性模型一样，Ridge用`fit()`方法完成拟合，并将模型系数`ω`存储在其`coef_`成员中：

In [15]:
from sklearn import linear_model
import numpy as np

reg = linear_model.Ridge(alpha= .5)
x = np.array([[0, 0], [0, 0], [1, 1]])
y = np.array([0, .1, 1])
reg.fit(x, y)

print(reg.coef_)
print(reg.intercept_)

[0.34545455 0.34545455]
0.1363636363636364


**岭回归的复杂度**
与普通最小二乘法的复杂度是相同的

**设置正则化参数：广义交叉验证**  
`RidgeCV`通过内置的关于的alpha参数的交叉验证来实现岭回归。该对象与GridSearchCV的使用方式相同，只是它默认为Generalized Cross-Validation（广义交叉验证GCV），这是一种有效的留一验证方法（LOO-CV）：

In [6]:
from sklearn import linear_model
reg = linear_model.RidgeCV(alphas=[0.1, 1.0, 10.0])
reg.fit([[0, 0], [0, 0], [1, 1]], [0, .1, 1])
reg.alpha_

0.1

指定cv属性的值将触发（通过GridSearchCV的）的交叉验证。例如，cv=10将出发10折的交叉验证，而不是广义交叉验证（GCV）

### 1.1.3 Lasso
`Lasso`是拟合稀疏系统的线性模型。它在一些情况下是有用的，因为它倾向于使用较少参数值的情况，有效地减少给定解决方案所依赖变量的数量。因此，Lasso及其变体是压缩感知领域的基础。子啊一定条件下，它可以恢复一组非零权重的精确集。  
在数学公式表达上，它由一个带有` \ell_1 `先验的正则项的线性模型组成。其最小化的目标函数是：
$$
\underset{w}{min\,} { \frac{1}{2n_{samples}} ||X w - y||_2 ^ 2 + \alpha ||w||_1}
$$
lasso estimate解决了加上罚项$ \alpha ||w||_1$的最小二乘法的最小化值，其中，α是一个常数，`||w||_1`是参数向量的`\ell_1`-norm范数   

`Lasso`类的实现使用了coordinate descent（坐标下降算法）来拟合系数。

In [7]:
from sklearn import linear_model
reg = linear_model.Lasso(alpha= 0.1)
reg.fit([[0, 0], [1, 1]], [0, 1])
reg.predict([[1, 1]])

array([0.8])

**设置正则化参数**  
`alpha`参数控制估计系数的稀疏度

- 使用交叉验证  
scikit-learn通过交叉验证来公开设置Lasso`alpha`参数的对象：`LassoCV`和`LassoLarsCV`。`LassoLarsCV`是基于下面将要提到的最小角回归算法 
